In [44]:
sc

In [45]:
mtaData = sc.textFile('dataset/clean-mta/clean-mta-data.csv',use_unicode=False)

In [46]:
def mapper1(index,data):
    # skip header row
    if index==0:
        data.next()
    import csv
    reader = csv.reader(data)
    for row in reader:
        event = row[4]
        numPeople = row[5].split('.')[0]
        yield (event, int(numPeople))
        
def mapper2(index,data):
    # skip header row
    if index==0:
        data.next()
    import csv
    reader = csv.reader(data)
    for row in reader:
        event = row[4]
        yield (event, 1)

# sum up all the entry by weather
# returns (event, total_people)
rdd1 = mtaData.mapPartitionsWithIndex(mapper1) \
                    .reduceByKey(lambda x,y: x+y)
rdd1.take(5)

# count how many days that have certain weather
# d = date; e = event; c = count;
# returns (event, (date, count))
rdd2 = mtaData.mapPartitionsWithIndex(mapper2) \
                    .reduceByKey(lambda x,y: x+y)
#                    .map(lambda ((d, e), c): (e, (d, c)))
rdd2.take(5)

# get average Entry in day for weather
# d = date; e = event; c = count; s = summation
# returns (event, average_peple_per_weather)
rdd3 = rdd1.join(rdd2) \
           .map(lambda (e, (s, (d, c))): ((e, s/c)))
rdd3.take(5)

[('Normal', ('08/26/2012', 29078)),
 ('Rain', ('04/21/2012', 31604)),
 ('Rain', ('12/23/2011', 29109)),
 ('Rain', ('10/12/2011', 31875)),
 ('Normal', ('07/28/2011', 31275))]